In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

In [2]:
import pandas as pd
import re
import pprint

## Connexion a spotify

In [3]:
CRED_PATH_SPOTIFY = "../credentials-spotify.json"

In [4]:
with open(CRED_PATH_SPOTIFY, 'r') as handle:
    data = json.load(handle)

In [5]:
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    **data
))

## Connexion a google

In [6]:
CREDENTIALS_PATH_GOOGLE = "../credentials-sheets.json"

In [7]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET = '1b75J-QTGrujSgF9r0_JPOKkcXAwzFVwpETOAyVBw8ak'

In [29]:
# Load service account credentials.
__credentials = Credentials.from_service_account_file(CREDENTIALS_PATH_GOOGLE, scopes=SCOPES)

# Creates Google Sheets API (v4/latest) service.
service = build('sheets', 'v4', credentials=__credentials)

In [30]:
values = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET, range='Notations').execute()['values']
headers = values.pop(0)

In [31]:
df = pd.DataFrame(values, columns=headers)

# DF preprocessing

In [32]:
df.set_index(['genre', 'sub_genre', 'artist', 'album', 'song'], inplace=True)
df = df.apply(lambda s: s.str.replace(",", "."))
df = df.apply(lambda s: pd.to_numeric(s, errors='coerce'))

In [33]:
df.describe()

,ntQ,ntG,ntV,ntR,ntS,ntGl,ntRx,ntC,ntL
count,1441.000000,706.000000,737.000000,217.000000,251.000000,113.000000,19.000000,20.000000,14.000000
mean,7.657009,8.072805,7.388738,7.733410,6.653586,7.484513,7.973684,6.435000,6.944286
std,1.340745,0.881173,1.343893,1.573626,1.945438,1.724949,2.288095,1.805044,3.057809
min,0.000000,1.000000,0.000000,0.100000,0.050000,0.000000,2.000000,0.000000,0.000000
25%,7.250000,7.500000,7.000000,7.300000,5.000000,7.000000,7.500000,5.950000,6.847500
50%,8.000000,8.250000,7.500000,8.000000,7.000000,7.500000,8.000000,6.625000,8.000000
75%,8.500000,8.750000,8.500000,8.700000,8.000000,8.500000,10.000000,7.375000,8.875000
max,10.000000,9.500000,9.750000,9.700000,10.000000,10.000000,10.000000,8.750000,9.330000


In [34]:
search = "artist:\"Black Sabbath\" track:\"Evil Woman\" album:\"Black Sabbath\""
# search = "artist:\"Black\%20Sabbath\"%20track:\"Evil%20Woman\"%20album:\"Black%20Sabbath\""
pprint.pp(spotify.search(search, market="FR"))

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3A%22Black+Sabbath%22+track%3A%22Evil+Woman%22+album%3A%22Black+Sabbath%22&type=track&market=FR&offset=0&limit=10',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5M52tdBnJaKSvOpJGz8mfZ'},
                                              'href': 'https://api.spotify.com/v1/artists/5M52tdBnJaKSvOpJGz8mfZ',
                                              'id': '5M52tdBnJaKSvOpJGz8mfZ',
                                              'name': 'Black Sabbath',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:5M52tdBnJaKSvOpJGz8mfZ'}],
                                 'external_urls': {'spotify': 'https://open.spotify.com/album/2T6jeELx5BqH4GMLObBy10'},
                                 'href': 'https://api.spotify.com/v1/albums/2T6jeE

In [48]:
def search_string(row):
    search = ""
    # artists
    artists = list(map(str.strip, row.artist.split(",")))
    if len(artists) > 1:
        sep = '" AND "'
        search += f"artist:\"{sep.join(artists)}\""
    else:
        search += f"artist:\"{artists[0]}\""
    # album
    if row.album != "N/A":
        search += f" album:\"{row.album}\""
    # track name
    search += f" track:\"{row.song}\""
    # dealing with "'""
    # sometimes it will work with the "'" and sometimes not
    if "'" in search:
        searches = [search, search.replace("'", "")]
    else:
        searches = [search]
    return searches

In [50]:
bad_formats = []
for idx, (_, content) in enumerate(df.index.to_frame().iterrows()):
    # search = f"artist:\"{content.artist}\" track:\"{content.song}\" album:\"{content.album}\""
    searches = search_string(content)
    bad_format = []
    for search in searches:
        try:
            res = spotify.search(search, market="FR")
            track = res['tracks']['items'][0]
        except IndexError as e:
            print(f"{search} not in spotify")
            bad_format.append(search)
        else:
            break
    else:
        bad_formats.append(bad_format)
        continue
    album = track['album']['name']
    name = track['name']
    artist = track['artists'][0]['name']
    id = track['id']
    print(f"{idx:<4}/{len(df)} : {id} {name} {artist} {album}")

0   /1633 : 0x5BKBAcia1th7UzcoHnD1 I Do, I Do, I Do, I Do, I Do ABBA Abba
1   /1633 : 2TxCwUlqaOH3TIyJqGgR91 Mamma Mia ABBA Abba
2   /1633 : 5pMmWfuL0FTGshYt7HVJ8P SOS ABBA ABBA Gold


3   /1633 : 798cuJeotvXP8UVa8GJPnD Knowing Me, Knowing You ABBA Arrival


4   /1633 : 4euAGZTszWPrriggYK0HG9 Lay All Your Love On Me ABBA Super Trouper
5   /1633 : 0J2p4KYdr6Mg4ET6JPlbe1 Super Trouper ABBA Super Trouper


6   /1633 : 3oEkrIfXfSh9zGnE7eBzSV The Winner Takes It All ABBA Super Trouper
7   /1633 : 5BckPAYcKEJuYs1eV1BHHe Take A Chance On Me ABBA The Album


8   /1633 : 3Dy4REq8O09IlgiwuHQ3sk Waterloo ABBA Waterloo


...

1617/1633 : 62jQ72vsUsLWLp6ZHFUJKr Fuck the System System Of A Down System Of A Down/Steal This Album
1618/1633 : 2HVCLZ6dMDelXhloO5sAjV Ego Brain System Of A Down Steal This Album!
1619/1633 : 6l17lK6a9LFqhOfh770wWg Thetawaves System Of A Down Steal This Album!


1620/1633 : 1Ip1XhR9OPvA9rPfm8uFZU Roulette System Of A Down System Of A Down/Steal This Album
1621/1633 : 39rHfrVqCX6A55GF7uOZSC 1612 Vulfpeck Fugue State
1622/1633 : 50R5Or6xMtvELGYfGUbVz4 3 on E Vulfpeck The Joy of Music. The Job of Real Estate.


1623/1633 : 5KboIqx30dztT6wmEVFn17 Baby I Don’t Know Oh Oh (feat. Charles Jones) Vulfpeck Mr Finish Line
1624/1633 : 1FwFVKRNX1IV4eEZSGGg5M Darwin Derby Vulfpeck Hill Climber
1625/1633 : 6egcs8AHSL7oPfq5TZ8w6n Test Drive - Instrumental Vulfpeck The Joy of Music. The Job of Real Estate.


1626/1633 : 7BHPGtpuuWWsvE7cCaMuEU Naive The Kooks Inside In / Inside Out
1627/1633 : 6E1gCX88g0FXzWMey57xlm 69 Parliament Medicaid Fraud Dogg
1628/1633 : 2l0h4aBFLp9HdoaNdCTlbW Pull the Plug Death Leprosy (Deluxe Reissue)


1629/1633 : 5noSkaAhiPUZZpKqgFWiLi Astral Tale Of Us Astral
1630/1633 : 15BQ7vEDv2LJuh8TxWIhtd The Passenger Iggy Pop Lust For Life
artist:"Duck Sauce" album:"Barbara Streisand" track:"Barbara Streisand" not in spotify
1632/1633 : 4MvGHDenL4t9JW1RHB4rK2 Gotta Go Home Boney M. Oceans Of Fantasy


In [52]:
bad_formats

[['artist:"Auracle" album:"City Slickers" track:"Bombs Away Ballet"'],
 ['artist:"Auracle" album:"City Slickers" track:"City of Penetrating Light"'],
 ['artist:"Auracle" album:"City Slickers" track:"Honey"'],
 ['artist:"Auracle" album:"City Slickers" track:"Little City Slickers"'],
 ['artist:"Auracle" album:"City Slickers" track:"Rotary Andy\'s Raggedy"',
  'artist:"Auracle" album:"City Slickers" track:"Rotary Andys Raggedy"'],
 ['artist:"Auracle" album:"City Slickers" track:"Sambanana"'],
 ['artist:"Auracle" album:"City Slickers" track:"Tied Shoes"'],
 ['artist:"B.J. Thomas" album:"Rain Drops Keep Fallin\' On My Head" track:"Rain Drops Keep Falling on My Head"',
  'artist:"B.J. Thomas" album:"Rain Drops Keep Fallin On My Head" track:"Rain Drops Keep Falling on My Head"'],
 ['artist:"Casiopea" album:"Crosspoint" track:"Galactic Funk"'],
 ['artist:"Casiopea" album:"Mint Jams" track:"Swear!"'],
 ['artist:"Creedence Clearwater Revival" album:"Cosmo\'s Factory" track:"I Heard It Through Th